# Raster - 2. Raster operations: From a global imagery layer to local suitability analysis.

This notebook will demonstrate a typical sequence of transformations for one factor in a suitability model


Run this cell to connect to your GIS and get started:

In [ ]:
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

from arcgis import GIS
from arcgis.raster import *
from arcgis.geometry import *
%matplotlib inline
import pandas as pd
import numpy as np
gis = GIS(username="izaslavsky_ucsd") 

In [ ]:
# if you plan to run costly opertions on AGOL, it is also a good idea to check available credits
iz = gis.users.get('izaslavsky_ucsd')
iz.availableCredits


## Typical steps for preparing a raster layer for suitability analysis:
 - retrieve the layer
 - specify area of interest (study area), e.g. by retrieving a named polygon and getting its extent in an explicit CRS
 - clip the layer to the area of interest
 - define categories to be shown in the output (these may be suitability classes) (make sure these classes actually exist)
 - define a colormap for these classes
 - remap the layer clip to these categories in this colormap
 
 
 You would need to keep track of the steps.
 I use a simple naming convention below:
 <code>
 *_geom : geometry of the study area
 *_ex   : extent of the study area
 *_n    : normalized raster, e.g. with a limited set of values that would be easy to interpret when combined with other data 
 *_b    : similar, but with 2 values only (binary)
 *\_n_c  : same, with applied colormap
 </code>

In [ ]:
# Step 1: Retrieing multispectral landsat from the Living Atlas, and referencing the correct layer

usa_mean_temperature = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/USA_Mean_Temperature/ImageServer", gis)

land_cover_vulnerability_2050 = ImageryLayer("https://env1.arcgis.com/arcgis/rest/services/Land_Cover_Vulnerability_2050/ImageServer",gis)

In [ ]:
# read about the layer on AGOL, if available

lcv = gis.content.get("20bfd812017e4bc1a241d2581c156bcd")
lcv

In [ ]:
# explore the layer via Python
land_cover_vulnerability_2050.properties

In [ ]:
land_cover_vulnerability_2050.key_properties()

In [ ]:
land_cover_vulnerability_2050.legend()

In [ ]:
# let's show the original map in a map widget
# Step 8: show it on a map, with a legend, to check if the result is as expected.
map1 = gis.map('San Diego, CA')
map1

In [ ]:
map1.zoom = 8
map1.legend.enabled = True
map1.content.add(land_cover_vulnerability_2050)

In [ ]:
# Step 2: figuring out the geometry of interest. 
# At the end of this step, we need to create a "study_area_extent" geometry, with an assigned coordinate system

# Here, we will use San Diego country as an example. We'll set it as the study area 

# 2a: retrieve the vector polygon layer and visualize it
counties_item = gis.content.search('USA Counties generalized', 'Feature Layer', outside_org=True)[4]
counties_lyr = counties_item.layers[0]


In [ ]:
counties_item

In [ ]:
# 2b: query the polygon layer, return its geometry, and set it as the study area. 

study_area_query = counties_lyr.query("FIPS='06073'", return_geometry=True)
study_area_geom= study_area_query.features[0].geometry
study_area_geom['spatialReference'] = study_area_query.spatial_reference
study_area_ex = Geometry(study_area_geom).geoextent  # or just 'extent'

# 2c: then create the extent as a dict from the returned tuple 
tup = (('xmin',study_area_ex[0]), ('ymin', study_area_ex[1]), ('xmax',study_area_ex[2]), ('ymax', study_area_ex[3]))
study_area_extent = dict(tup)

# 2d: to be correcter, let's also add CRS:
crs = "{'latestwkid':3857, 'wkid':102100}"
study_area_extent.update({'spatialReference':{'latestwkid':3857, 'wkid':102100}})


In [ ]:
#2e: check the geometry
study_area_geom

In [ ]:
# Step 3: clip lcv raster (from step 1) to SD geometry (from step 2):
lcv_study_area = clip(raster = land_cover_vulnerability_2050, geometry = study_area_geom)

lcv_study_area.legend()  # notice that the clip's legend is different


In [ ]:
#  Step 3a: Let's figure out the range of values in the clipped area:
histograms = lcv_study_area.compute_histograms(geometry=study_area_geom)

In [ ]:
histogram = histograms['histograms'][0]  # Adjust index for specific bands if needed
min_value = histogram['min']
max_value = histogram['max']
print(f"Min Value: {min_value}, Max Value: {max_value}")

In [ ]:
# Alternatively, just look these up in layer properties

print(lcv_study_area.properties.minValues)
print(lcv_study_area.properties.maxValues)

In [ ]:
# Step 4: decide how to present the clipped raster by categories. Here, we define input range for lcv in 8 categories
# In a suitability model, it is typically easier to work with a limited set of values
# which are at the same scale as other rasters (often referred to as "normalization")

lcv_input_ranges_n = [0.0, 100,
                      100, 200,
                      200, 300, 
                      300, 400,
                      400, 500,
                      500, 600,
                      600, 700,
                      700, 1000]


# Step 5: Create a colormap values for the 8 raster categories defined in step 4 (the values will be from 0 to 7):
clrmap8 =  [[7, 242, 85, 0], [6, 250, 142, 0], [5, 255, 195, 0], [4, 255, 255, 0], [3, 197, 219, 0],  
          [2, 139, 181, 0], [1, 86, 148, 0], [0, 38, 115, 0]]


# Step 6: remap the raster to the 8 categories, and apply the raster extent
lcv_n = remap(raster = lcv_study_area,
               input_ranges = lcv_input_ranges_n,
               output_values = [i for i in range(8)],
               astype = 'U8')
lcv_n.extent= study_area_extent


In [ ]:
# Step 7: apply the colormap (from step 5) to the remapped raster (from step 6). 
# This will generate a normalized color view of the suitability factor


lcv_n_c = colormap(raster=lcv_n, colormap=clrmap8)


In [ ]:
# Step 8: show it on a map, with a legend, to check if the result is as expected.
map2 = gis.map('San Diego, CA')
map2

In [ ]:
map2.zoom = 8
map2.legend.enabled = True
map2.content.add(lcv_n_c)

In [ ]:
import graphviz
lcv_n_c.draw_graph()

# this is a typical preparation sequence

## A few rasters you can use as sources of data

In [ ]:


usa_mean_temperature = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/USA_Mean_Temperature/ImageServer", gis)
usa_landcover_gap = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/USA_Landcover_GAP/ImageServer", gis)
usa_slope = ImageryLayer("https://landscape3.arcgis.com/arcgis/rest/services/Landscape_Modeler/USA_Slope/ImageServer", gis)

usa_wetlands = ImageryLayer('https://landscape11.arcgis.com/arcgis/rest/services/USA_Wetlands/ImageServer',gis)

usa_soil_erosion = ImageryLayer('https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erosion_Class/ImageServer',gis)

global_air_pollution = ImageryLayer('https://sedac.ciesin.columbia.edu/arcgis/rest/services/sedac/sdei_global_annual_avg_pm2_5_2001_2010_image_service/ImageServer',gis)

global_GLDAS_evapotranspiration = ImageryLayer('https://earthobs2.arcgis.com/arcgis/rest/services/GLDAS_Evapotranspiration/ImageServer',gis)

usa_federal_lands = ImageryLayer('https://landscape10.arcgis.com/arcgis/rest/services/USA_Federal_Lands/ImageServer',gis)

usa_soil_erodibility = ImageryLayer('https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erodibility_Factor/ImageServer',gis)

# more available imagery layers to check out: see landscape1.arcgis.com ... landscape13.arcgis.com


# see functions at https://pro.arcgis.com/en/pro-app/help/data/imagery/calculator-function.htm 